## 配置运行环境

In [ ]:
# 重启
#!kill -9 -1
# 安装包
# !pip install packagename

In [1]:
# 挂载谷歌硬盘 从drive导入数据
from google.colab import drive
drive.mount('/content/drive')
# 创建数据文件夹
!mkdir /content/dataset
!mkdir .kaggle
# 复制文件到本地
# !cp /content/drive/My\ Drive/Dataset/filename /content/dataset/filename
# 切换当前目录
# %cd /content/dataset/
# 解压缩文件
# !unzip filename

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
mkdir: cannot create directory ‘/content/dataset’: File exists
mkdir: cannot create directory ‘.kaggle’: File exists


In [2]:
# 从kaggle导入数据
!cp /content/drive/My\ Drive/Dataset/kaggle.json ~/.kaggle/kaggle.json
!kaggle config set -n path -v{/content}
# !kaggle datasets list
!kaggle datasets list -s ctr #搜索指定数据集
!kaggle datasets download -d wuyingwen06/avazu-ctr-train  -p /content/dataset
!kaggle datasets download -d wuyingwen06/avazu-ctr  -p /content/dataset
%cd /content/dataset/
!unzip avazu-ctr-train.zip
!unzip avazu-ctr.zip

- path is now set to: {/content}
ref                                                                  title                                            size  lastUpdated          downloadCount  
-------------------------------------------------------------------  ----------------------------------------------  -----  -------------------  -------------  
sohier/recommender-click-logs-sowiport                               Recommender Click Logs- Sowiport                109MB  2020-05-12 04:19:45            726  
atirpetkar/avazu-ctr                                                 avazu ctr                                         1GB  2018-02-25 03:11:45            539  
animeshgoyal9/predict-click-through-rate-ctr-for-a-website           Predict Click through rate (CTR) for a website   23MB  2019-06-20 20:48:31            327  
wuyingwen06/avazu-ctr-train                                          avazu_ctr_train                                   1GB  2019-04-15 08:29:23             86  
s

## 数据预处理

In [ ]:
import pandas as pd
import numpy as np
import gc

train=pd.read_csv('train.csv')
print(train.columns)

test=pd.read_csv('test.csv')
print(test.columns)

print(train.shape)
print(test.shape)

In [ ]:
# 通过设置字符类型来降低大小
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [ ]:
train = pd.concat([train, test], ignore_index=True, sort=False)

train=reduce_mem_usage(train)

train['time']=pd.to_datetime(train['hour'],format='%y%m%d%H')
train['day']=train['time'].dt.day
train['hour']=train['time'].dt.hour

train['user']=train['device_id'].astype(str)+ '_' + train['device_ip'].astype(str)+ '_' +train['device_model'].astype(str)
train['app']=train['app_id'].astype(str)+ '_' + train['app_domain'].astype(str)+ '_' +train['app_category'].astype(str)
train['site']=train['site_id'].astype(str)+ '_' + train['site_domain'].astype(str)+ '_' +train['site_category'].astype(str)

In [ ]:
#选27-31日的数据训练
train = train[(train.day==27)|(train.day==28)|(train.day==29)|(train.day==30)|(train.day==31)]

matrix=train
del train,test
gc.collect()